# Questão 5

Para a arealização da questão 5, o primeiro passo foi verificar qual seria o número de steps exigidos por cada método para resultar um um tempo aproximado de impressão. Para isto, foi utilizado o script abaixo, no qual se ajustou manualmente o número de iterações para o treinamento por cada técnica reproduzindo o script diversas vezes. Ao final, regulamos o tempo de treinamento de todas em cerca de 10,5 segundos, pois este era o tempo em que as melhores técnicas começavam a saturar seus resultados em 100% de recompensa nos testes (200/200 em 5 vezes). Foi utilizado um treinamento de 3200 steps.

Ainda utilizando o script abaixo, verificamos o desempenho médio de cada técnica no teste. Sendo 5 recompensas de até 200 pontos para cada uma, a técnica de melhor desempenho foi a Deep Q Learning (DQN), com 196,8 steps em média antes de deixar o pêndulo cair (em 4 tentativas o pêndulo sequer caiu, com a contagem saturando em 200). 

A segunda melhor técnica foi a Cross-Entropy Method (CEM), que obteve 192,4 steps em média, tendo treinado com 11000 (o maior número de steps do grupo). O tempo de execução de seus step é rápido, mas ela exige muitos steps para melhorar seu desempenho.

A terceira melhor técnica foi a Duel DQN, que atingiu 165,6 de desempenho após um treinamento de apenas 2900 steps (o menor do grupo). Esta técnica não necessita de tantos steps para ser treinada, mas estes são de lenta execução.

O pior resultado fica com a State-ActionReward-State-Action, que utilizou treinamentos de 4500 steps e obteve desempenho médio de 24,2 antes de falhar nos testes, deixando o pêndulo cair do carrinho muito rapidmente em relação à outra.

Vale ressaltar que, quando aumentado o número de steps dos treinamentos, todas as técnicas atingiram desempenho médio igual ou próximo a 100%, indicando que elas podem cumprir seu objetivo, demandando apenas diferentes tempos de treino para tal.

In [1]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.memory import EpisodeParameterMemory

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

import time

ENV_NAME = 'CartPole-v0'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

nb_actions = env.action_space.n




# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

obs_dim = env.observation_space.shape[0]

# Option 1 : Simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))

# Option 2: deep network
# model = Sequential()
# model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(nb_actions))
# model.add(Activation('softmax'))


print(model.summary())


# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = EpisodeParameterMemory(limit=1000, window_length=1)

cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
               batch_size=50, nb_steps_warmup=2000, train_interval=50, elite_frac=0.05)
cem.compile()



# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# SARSA does not require a memory.
policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=10, policy=policy)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])



# Next, we build a very simple model regardless of the dueling architecture
# if you enable dueling network in DQN , DQN will build a dueling network base on your model automatically
# Also, you can build a dueling network by yourself and turn off the dueling network in DQN.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions, activation='linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
# enable the dueling network
# you can specify the dueling_type to one of {'avg','max','naive'}
duel = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               enable_dueling_network=True, dueling_type='avg', target_model_update=1e-2, policy=policy)
duel.compile(Adam(lr=1e-3), metrics=['mae'])



print("TEMPO: ", time.time())
# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
duel.fit(env, nb_steps=2900, visualize=True, verbose=2)

print("TEMPO: ", time.time())
# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
dqn.fit(env, nb_steps=3200, visualize=True, verbose=2)

print("TEMPO: ", time.time())
# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
cem.fit(env, nb_steps=11000, visualize=True, verbose=2)

print("TEMPO: ", time.time())
# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
sarsa.fit(env, nb_steps=4500, visualize=True, verbose=2)
print("TEMPO: ", time.time())




# After training is done, we save the final weights.
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)


print("\n\nDQN\n\n")
# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=True)


# After training is done, we save the best weights.
cem.save_weights('cem_{}_params.h5f'.format(ENV_NAME), overwrite=True)

print("\n\CEM\n\n")
# Finally, evaluate our algorithm for 5 episodes.
cem.test(env, nb_episodes=5, visualize=True)



# After training is done, we save the final weights.
sarsa.save_weights('sarsa_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

print("\n\SARSA\n\n")
# Finally, evaluate our algorithm for 5 episodes.
sarsa.test(env, nb_episodes=5, visualize=True)




# After training is done, we save the final weights.
duel.save_weights('duel_dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

print("\n\DUEL\n\n")
# Finally, evaluate our algorithm for 5 episodes.
duel.test(env, nb_episodes=5, visualize=True)


Using TensorFlow backend.
W0628 07:54:50.469259 140545833047872 deprecation_wrapper.py:119] From /home/patrick/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0628 07:54:50.484366 140545833047872 deprecation_wrapper.py:119] From /home/patrick/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0628 07:54:50.497800 140545833047872 deprecation_wrapper.py:119] From /home/patrick/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0628 07:54:50.672117 140545833047872 deprecation_wrapper.py:119] From /home/patrick/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please u

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                80        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
__________

W0628 07:54:50.804264 140545833047872 deprecation_wrapper.py:119] From /home/patrick/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 10        
_________________________________________________________________
activation_5 (Activation)    (None, 2)                 0         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                80        
_________________________________________________________________
activatio

/home/patrick/anaconda3/lib/python3.7/site-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/home/patrick/anaconda3/lib/python3.7/site-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


...

  695/4500: episode: 47, duration: 0.026s, episode steps: 11, steps per second: 429, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.091 [0.000, 1.000], mean observation: 0.146 [-1.731, 2.783], loss: 8.395165, mean_absolute_error: 5.950137, mean_q: 11.384029
  706/4500: episode: 48, duration: 0.027s, episode steps: 11, steps per second: 402, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.091 [0.000, 1.000], mean observation: 0.124 [-1.789, 2.829], loss: 8.435202, mean_absolute_error: 5.932163, mean_q: 11.424528
  715/4500: episode: 49, duration: 0.020s, episode steps: 9, steps per second: 444, episode reward: 9.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.000 [0.000, 0.000], mean observation: 0.141 [-1.789, 2.804], loss: 9.081702, mean_absolute_error: 5.743052, mean_q: 11.090290
  727/4500: episode: 50, duration: 0.026s, episode steps: 12, steps per second: 467, episode reward: 12.000, mean reward: 1.000 [1.000, 1.000], mea

 1191/4500: episode: 80, duration: 0.046s, episode steps: 19, steps per second: 412, episode reward: 19.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.579 [0.000, 1.000], mean observation: -0.065 [-1.478, 0.961], loss: 3.900349, mean_absolute_error: 7.030389, mean_q: 13.109353
 1202/4500: episode: 81, duration: 0.024s, episode steps: 11, steps per second: 462, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.727 [0.000, 1.000], mean observation: -0.104 [-1.868, 1.189], loss: 8.600678, mean_absolute_error: 7.406077, mean_q: 13.398121
 1213/4500: episode: 82, duration: 0.024s, episode steps: 11, steps per second: 451, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.818 [0.000, 1.000], mean observation: -0.159 [-2.301, 1.327], loss: 7.953047, mean_absolute_error: 7.302922, mean_q: 13.248173
 1231/4500: episode: 83, duration: 0.039s, episode steps: 18, steps per second: 463, episode reward: 18.000, mean reward: 1.000 [1.000, 1.000]

 1863/4500: episode: 110, duration: 0.072s, episode steps: 31, steps per second: 432, episode reward: 31.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.484 [0.000, 1.000], mean observation: -0.099 [-0.800, 0.398], loss: 5.528775, mean_absolute_error: 9.950999, mean_q: 18.657060
 1877/4500: episode: 111, duration: 0.029s, episode steps: 14, steps per second: 481, episode reward: 14.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: 0.098 [-0.741, 1.298], loss: 11.911693, mean_absolute_error: 9.966255, mean_q: 18.435518
 1903/4500: episode: 112, duration: 0.066s, episode steps: 26, steps per second: 392, episode reward: 26.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.500 [0.000, 1.000], mean observation: -0.101 [-1.207, 0.738], loss: 6.307186, mean_absolute_error: 9.793403, mean_q: 18.342842
 1944/4500: episode: 113, duration: 0.088s, episode steps: 41, steps per second: 466, episode reward: 41.000, mean reward: 1.000 [1.000, 1.

 2733/4500: episode: 143, duration: 0.036s, episode steps: 14, steps per second: 390, episode reward: 14.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.571 [0.000, 1.000], mean observation: -0.083 [-1.706, 1.201], loss: 8.123520, mean_absolute_error: 9.427062, mean_q: 17.745267
 2742/4500: episode: 144, duration: 0.023s, episode steps: 9, steps per second: 384, episode reward: 9.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.778 [0.000, 1.000], mean observation: -0.128 [-1.970, 1.212], loss: 10.684997, mean_absolute_error: 9.329206, mean_q: 17.194337
 2755/4500: episode: 145, duration: 0.042s, episode steps: 13, steps per second: 310, episode reward: 13.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.692 [0.000, 1.000], mean observation: -0.077 [-2.129, 1.397], loss: 6.249128, mean_absolute_error: 8.701442, mean_q: 16.126359
 2787/4500: episode: 146, duration: 0.068s, episode steps: 32, steps per second: 472, episode reward: 32.000, mean reward: 1.000 [1.000, 1.0

 3791/4500: episode: 174, duration: 0.048s, episode steps: 20, steps per second: 420, episode reward: 20.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.550 [0.000, 1.000], mean observation: -0.065 [-1.225, 0.622], loss: 13.408300, mean_absolute_error: 13.564146, mean_q: 25.727165
 3826/4500: episode: 175, duration: 0.076s, episode steps: 35, steps per second: 462, episode reward: 35.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.457 [0.000, 1.000], mean observation: -0.096 [-1.147, 0.629], loss: 7.750299, mean_absolute_error: 12.678521, mean_q: 24.407891
 3841/4500: episode: 176, duration: 0.061s, episode steps: 15, steps per second: 247, episode reward: 15.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.467 [0.000, 1.000], mean observation: 0.093 [-0.418, 1.007], loss: 18.974102, mean_absolute_error: 13.189904, mean_q: 24.509106
 3911/4500: episode: 177, duration: 0.187s, episode steps: 70, steps per second: 374, episode reward: 70.000, mean reward: 1.000 [1.000